In [1]:
import pandas as pd
complete_data = pd.read_csv('complete_data-edit.csv')
complete_data['Outage'] = complete_data['Category'].apply(lambda x: 0 if pd.isna(x) else 1)
relevant_columns = [
    'Date', 'Branch', 'Location', 'Feeder', 'Category', 'Fault', 'Areas Affected', 'Peak Generation',
    'Daily Energy Generation', 'Lowest Energy Generation', 'Daily Energy Sent', '6:00 Generation',
    'Highest System Frequency', 'Lowest System Frequency', 'Highest Voltage Recorded', 'Lowest Voltage Recorded',
    'temp', 'humidity',
    'precip', 'precipprob', 'precipcover', 'windgust', 'windspeed', 'winddir', 'sealevelpressure',
    'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex',
    'conditions', 'description', 'icon', 'feelslike', 'Outage'
]

initial_data = complete_data[relevant_columns]

In [2]:
# Create an empty list to store the values for 'OutageNextDay'. Since we want to predict if an outage exists 
df = initial_data
df['Date'] = pd.to_datetime(df['Date'])

# Initialize an empty list to store 'OutageNextDay' values
outage_next_day = []

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    # Get the current date and outage status
    current_date = row['Date']
    current_outage = row['Outage']
    
    # Find the index of the next row with the same date
    next_index = index + 1
    while next_index < len(df) and df.loc[next_index, 'Date'] == current_date:
        next_index += 1
    
    # If the next row exists and has the same date, record its outage status
    if next_index < len(df):
        next_outage = df.loc[next_index, 'Outage']
        outage_next_day.append(next_outage)
    else:
        outage_next_day.append(None)  # No next day outage
    
# Assign the 'OutageNextDay' values to the DataFrame
df['OutageNextDay'] = outage_next_day
df.loc[df['Date'] == '2023-12-02', 'OutageNextDay'] = 0

/var/folders/dp/fn2v5z3x7pj63xx0j5wvqjvm0000gp/T/ipykernel_32375/2117579228.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])
/var/folders/dp/fn2v5z3x7pj63xx0j5wvqjvm0000gp/T/ipykernel_32375/2117579228.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['OutageNextDay'] = outage_next_day


In [3]:
# Convert the 'OutageNextDay' column to integer type
df['OutageNextDay'] = df['OutageNextDay'].astype(int)

/var/folders/dp/fn2v5z3x7pj63xx0j5wvqjvm0000gp/T/ipykernel_35712/2787134435.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['OutageNextDay'] = df['OutageNextDay'].astype(int)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

relevant_data = df
X = relevant_data #except outagenextday
y = relevant_data['OutageNextDay']

'''
drops = ['Date', 'Branch', 'Location', 'Feeder', 'Category', 'Fault', 'Areas Affected', 'name', 'datetime',
        'preciptype', 'sunrise', 'sunset', 'conditions', 'description', 'icon', 'stations',
         'OutageNextDay', 'Next_Day_Branch', 'severerisk', 'cloudcover', 'snow', 'snowdepth', 'windgust'] 
'''
drops = ['Date', 'Branch', 'Location', 'Feeder', 'Category', 'Fault', 'Areas Affected',
         'conditions', 'description', 'icon',
         'OutageNextDay', 'cloudcover', 'windgust'] 
X = X.drop(columns=drops)
# Fill NaN or empty values in 'snow' and 'snowdepth' columns with 0.0
#X['snow'] = X['snow'].fillna(0.0)
#X['snowdepth'] = X['snowdepth'].fillna(0.0)

# Get indices of NaN values in X
dropped_indices = X[X.isnull().any(axis=1)].index

# Drop NaN values from X and y
X = X.dropna()
y.drop(dropped_indices, inplace=True)

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
import numpy as np

# Initialize empty lists to store metrics
precision_scores_0 = []
recall_scores_0 = []
f1_scores_0 = []
precision_scores_1 = []
recall_scores_1 = []
f1_scores_1 = []
auc_scores = []

for random_state in range(20):
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    
    # Create and fit the Random Forest Classifier
    rf_model = RandomForestClassifier(n_estimators = 1000, min_samples_split=13, class_weight = {0: 0.23, 1: 0.66},
                                      max_depth = 15, random_state=42)
    rf_model.fit(X_train, y_train)
    
    # Predict on test set
    y_pred = rf_model.predict(X_test)

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Accumulate precision, recall, and F1-score for class 0 and class 1
    precision_scores_0.append(report['0']['precision'])
    recall_scores_0.append(report['0']['recall'])
    f1_scores_0.append(report['0']['f1-score'])
    precision_scores_1.append(report['1']['precision'])
    recall_scores_1.append(report['1']['recall'])
    f1_scores_1.append(report['1']['f1-score'])
    
    # Calculate AUC
    auc = roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1])
    auc_scores.append(auc)

# Calculate average metrics for class 0
avg_precision_0 = np.mean(precision_scores_0)
avg_recall_0 = np.mean(recall_scores_0)
avg_f1_0 = np.mean(f1_scores_0)

# Calculate average metrics for class 1
avg_precision_1 = np.mean(precision_scores_1)
avg_recall_1 = np.mean(recall_scores_1)
avg_f1_1 = np.mean(f1_scores_1)

# Calculate average AUC
avg_auc = np.mean(auc_scores)

# Print average metrics for class 0
print("Average metrics for Class 0:")
print(f'Average Precision: {avg_precision_0:.2f}')
print(f'Average Recall: {avg_recall_0:.2f}')
print(f'Average F1-Score: {avg_f1_0:.2f}')

# Print average metrics for class 1
print("\nAverage metrics for Class 1:")
print(f'Average Precision: {avg_precision_1:.2f}')
print(f'Average Recall: {avg_recall_1:.2f}')
print(f'Average F1-Score: {avg_f1_1:.2f}')

# Print average AUC
print("\nAverage AUC:")
print(f'Average AUC: {avg_auc:.2f}')

Average metrics for Class 0:
Average Precision: 0.78
Average Recall: 0.80
Average F1-Score: 0.79

Average metrics for Class 1:
Average Precision: 0.70
Average Recall: 0.68
Average F1-Score: 0.69

Average AUC:
Average AUC: 0.82


In [7]:
from joblib import dump

# Assuming your logistic regression model is named lin_reg
dump(rf_model, 'Outage Prediction Model.joblib')

['Outage Prediction Model.joblib']